In [4]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-4-5-sonnet"

In [5]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [6]:
# Function to generate a new dataset
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [7]:
# Generate the dataset and write it to 'dataset.json'
dataset = generate_dataset()
with open("dataset2.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [8]:
# Function to grade a test case + output using a model
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [9]:
# Passes a test case into Claude
def run_prompt(test_case):
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""

    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [10]:
# Function to execute a single test case and grade the output
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning,
    }

In [12]:
from statistics import mean


def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [13]:
with open("dataset2.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 7.333333333333333


In [14]:
print(json.dumps(results, indent=2))

[
  {
    "output": "I'll create a regular expression that validates AWS S3 bucket names according to all the specified rules.\n\n```regex\n^(?!.*\\.\\.)(?!.*\\.$)(?!^\\.)([a-z0-9])([a-z0-9.-]{1,61})([a-z0-9])$\n```\n\n**Explanation of the regex components:**\n\n1. `^` - Start of string\n2. `(?!.*\\.\\.)` - Negative lookahead: no consecutive periods anywhere\n3. `(?!.*\\.$)` - Negative lookahead: doesn't end with a period\n4. `(?!^\\.)` - Negative lookahead: doesn't start with a period\n5. `([a-z0-9])` - Must start with a lowercase letter or number (1 character)\n6. `([a-z0-9.-]{1,61})` - Middle characters: lowercase letters, numbers, hyphens, or periods (1-61 characters)\n7. `([a-z0-9])` - Must end with a lowercase letter or number (1 character)\n8. `$` - End of string\n\nThis ensures:\n- **Total length: 3-63 characters** (1 + 1-61 + 1 = 3-63)\n- **Only lowercase letters, numbers, hyphens, and periods**\n- **Starts with a letter or number**\n- **Ends with a letter or number**\n- **No 